# Notebook 2: Modelling and Metrics

In [1]:
from boruta import BorutaPy
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

import seaborn as sns
import matplotlib.pyplot as plt

## Load

In [2]:
# Loading train and test.
X_train = pd.read_csv('data/X_train.csv').drop(columns='ID')
X_test = pd.read_csv('data/X_test.csv').drop(columns='ID')
y_train = pd.read_csv('data/y_train.csv')
y_test = pd.read_csv('data/y_test.csv')

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(23988, 38) (5998, 38) (23988, 1) (5998, 1)


In [3]:
# Reshape arrays into a flat format.
y_train = np.ravel(y_train)
y_test = np.ravel(y_test)

In [4]:
# Import original features name.
with open('data/original_features.txt') as f:
    list_orig_feat = f.readlines()

list_orig_feat = [i.strip() for i in list_orig_feat]
list_orig_feat

['LIMIT_BAL',
 'SEX',
 'EDUCATION',
 'MARRIAGE',
 'AGE',
 'PAY_0',
 'PAY_2',
 'PAY_3',
 'PAY_4',
 'PAY_5',
 'PAY_6',
 'BILL_AMT1',
 'BILL_AMT2',
 'BILL_AMT3',
 'BILL_AMT4',
 'BILL_AMT5',
 'BILL_AMT6',
 'PAY_AMT1',
 'PAY_AMT2',
 'PAY_AMT3',
 'PAY_AMT4',
 'PAY_AMT5',
 'PAY_AMT6']

## Model

### Validation

In [5]:
# Create a dataframe that will hold all experiments details and scores.
df_metric = pd.DataFrame(columns=['algorithm', 'accuracy', 'recall', 'precision', 'f2score', 'roc_auc'])
df_metric

,algorithm,accuracy,recall,precision,f2score,roc_auc


In [6]:
def optimal_threshold(y_true, y_pred_proba):
    '''Find the threshold that approximates TPR and FPR.
    Previous experiments:
    - TPR == FPR: fail (converge could happen anywhere, not necessarily at optimal threshold)
    - max f2score: fail (unnecessary increased of False Positive)
    - max(TPR - FPR): success (it b)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred_proba)

    df = pd.DataFrame({
        'tpr': tpr,
        'fpr': fpr,
        'threshold': thresholds
    })
    df['fx'] = df['tpr'] - df['fpr']
    best_threshold = df.loc[df['fx'] == df['fx'].max(), 'threshold'].values[0]
    return best_threshold

In [7]:
class FeatureSelection(BaseEstimator, TransformerMixin):
    
    def __init__(self, max_iter, estimator):
        '''Perform feature selection inside each fold.
        Insights: https://insightsimaging.springeropen.com/articles/10.1186/s13244-021-01115-1
        '''
        self.max_iter=max_iter
        self.estimator=estimator

    def fit(self, X, y):
        # Since RandomizedSearchCV uses only 'fit' and no 'transform', this method will also use only 'fit'.
        self.list_feat = list(X.columns)
        boruta = BorutaPy(
            estimator=self.estimator,
            n_estimators='auto',
            max_iter=self.max_iter
            )
        boruta.fit(np.array(X), np.array(y))
        self.boruta = boruta
        
        feat_selection = [i for i, j in zip(self.list_feat, self.boruta.support_) if j == True]  
        self.feat_selection = feat_selection
        return self

    def transform(self, X):
        return X[self.feat_selection]


# Check
#check = FeatureSelection(max_iter=10, estimator=RandomForestClassifier(max_depth=5, random_state=52, n_estimators=100))
#check.fit(X_train, y_train)

In [8]:
# Create nested stratified cross validation with hyperparameter optimization process:
def pipeline_train_model(X, y, k, feat_selection_iter, random_search_iter, model, params_dim):

    list_acc = []
    list_rec = []
    list_prec = []
    list_f2 = []
    list_roc_auc = []

    # Outer cross validation.
    kfold = StratifiedKFold(n_splits=k, shuffle=True, random_state=52)
    for train_index, validation_index in kfold.split(X, y):
        
        X_train = X.loc[train_index]
        X_valid = X.loc[validation_index]
        y_train = y[train_index]
        y_valid = y[validation_index]
        
        # Inner cross validation encapsulated in pipeline.
        feature_selection = FeatureSelection(max_iter=feat_selection_iter, estimator=RandomForestClassifier(max_depth=5, random_state=52, n_estimators=100))
        cv = RandomizedSearchCV(estimator=model,
                                param_distributions=params_dim,
                                cv=3,
                                n_iter=random_search_iter,
                                random_state=52)

        pipe = Pipeline(
            [
                ('feature_selection', feature_selection),
                ('cv', cv)
            ]
        )
        pipe.fit(X_train, y_train)              
        best_params = pipe['cv'].best_params_
        best_feat = pipe['feature_selection'].feat_selection
        
        # Train and validate model.
        model.set_params(**best_params)
        model.fit(X_train[best_feat], y_train)
        y_pred_proba = model.predict_proba(X_valid[best_feat])[:, 1]
        
        # Classify the probability conditional to train sample %credit default.
        best_threshold = optimal_threshold(y_valid, y_pred_proba)
        y_pred = [1 if i >= best_threshold else 0 for i in y_pred_proba]

        list_acc.append(metrics.accuracy_score(y_valid, y_pred))
        list_rec.append(metrics.recall_score(y_valid, y_pred))
        list_prec.append(metrics.precision_score(y_valid, y_pred))
        list_f2.append(metrics.fbeta_score(y_valid, y_pred, beta=2))
        list_roc_auc.append(metrics.roc_auc_score(y_valid, y_pred_proba))
        
    # Calculate average for every metric.
    avg_acc = np.mean(list_acc)
    avg_rec = np.mean(list_rec)
    avg_prec = np.mean(list_prec)
    avg_f2 = np.mean(list_f2)
    avg_roc_auc = np.mean(list_roc_auc)

    return [avg_acc, avg_rec, avg_prec, avg_f2, avg_roc_auc]

'''
#(testing)
model = LogisticRegression(random_state=52)
params_dim = {
    'max_iter':[300, 400, 500],
    'solver':['liblinear', 'saga'],
    'penalty': ['l1', 'l2']
}

check = pipeline_train_model(X_train, y_train, k=3, feat_selection_iter=10, random_search_iter=10, model=model, params_dim=params_dim)
'''

"\n#(testing)\nmodel = LogisticRegression(random_state=52)\nparams_dim = {\n    'max_iter':[300, 400, 500],\n    'solver':['liblinear', 'saga'],\n    'penalty': ['l1', 'l2']\n}\n\ncheck = pipeline_train_model(X_train, y_train, k=3, feat_selection_iter=10, random_search_iter=10, model=model, params_dim=params_dim)\n"

### M1: Baseline

In [9]:
# Each row prediction is based on training dataset %default: 1 (default) or 0 (non-default).
m1_target_prob = np.unique(y_train, return_counts=True)[1][1] / len(y_train)
print(f"Training sample default%: {m1_target_prob}")

y_pred_m1 = np.random.uniform(0, 1, len(y_train))
y_pred_m1 = [1 if i >= m1_target_prob else 0 for i in y_pred_m1]

print(f"Predicted sample default%: {np.unique(y_pred_m1, return_counts=True)[1][0] / len(y_pred_m1)}")

Training sample default%: 0.2213189928297482
Predicted sample default%: 0.2221110555277639


In [15]:
# Saving results in df_metric.
df_metric.loc['m1_baseline'] = ['probability_in_sample',
                                metrics.accuracy_score(y_train, y_pred_m1),
                                metrics.recall_score(y_train, y_pred_m1),
                                metrics.precision_score(y_train, y_pred_m1),
                                metrics.fbeta_score(y_train, y_pred_m1, beta=2),
                                None]

df_metric

,algorithm,accuracy,recall,precision,f2score,roc_auc
m1_baseline,probability_in_sample,0.346131,0.780185,0.221972,0.519100,NaN
m2_logregression,LogisticRegression,0.749750,0.615751,0.453154,0.573856,0.749058
m3_decisiontree,DecisionTree,0.771969,0.572225,0.492438,0.552066,0.757373
m4_randomforest,RandomForestClassifier,0.764132,0.617635,0.477426,0.582166,0.779022


### M2: Log Regression

In [11]:
# Log Regression with full features.
model = LogisticRegression(random_state=52)
params_dim = {
    'max_iter':[300, 400, 500],
    'solver':['liblinear', 'saga'],
    'penalty': ['l1', 'l2']
}

score = pipeline_train_model(X_train, y_train, k=3, feat_selection_iter=10, random_search_iter=10, model=model, params_dim=params_dim)

# Saving results in df_metric.
df_metric.loc['m2_logregression'] = ['LogisticRegression', score[0], score[1], score[2], score[3], score[4]]
df_metric

c:\Users\Rafael\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Rafael\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Rafael\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Rafael\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Rafael\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means

,algorithm,accuracy,recall,precision,f2score,roc_auc
m1_baseline,NaN,0.346131,0.780185,0.221972,0.519100,NaN
m2_logregression,LogisticRegression,0.749750,0.615751,0.453154,0.573856,0.749058


### M3: Decision Tree Classifier

In [12]:
# Single decision tree classifier.
model = DecisionTreeClassifier(random_state=52)
params_dim = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_depth': [5, 15, 30, 200],
    'min_samples_split': [2, 50, 100, 200],
    'min_samples_leaf': [1, 50, 100, 200]
}

score = pipeline_train_model(X_train, y_train, k=3, feat_selection_iter=10, random_search_iter=10, model=model, params_dim=params_dim)

# Saving results in df_metric.
df_metric.loc['m3_decisiontree'] = ['DecisionTree', score[0], score[1], score[2], score[3], score[4]]
df_metric

,algorithm,accuracy,recall,precision,f2score,roc_auc
m1_baseline,NaN,0.346131,0.780185,0.221972,0.519100,NaN
m2_logregression,LogisticRegression,0.749750,0.615751,0.453154,0.573856,0.749058
m3_decisiontree,DecisionTree,0.771969,0.572225,0.492438,0.552066,0.757373


### M4: Random Forest Classifier

In [13]:
# Single decision tree classifier.
model = RandomForestClassifier(random_state=52)
params_dim = {
    'n_estimators': [100, 250, 500],
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_depth': [5, 15, 30],
    'min_samples_split': [2, 50, 100, 200],
    'min_samples_leaf': [1, 50, 100, 200]
}

score = pipeline_train_model(X_train, y_train, k=3, feat_selection_iter=10, random_search_iter=10, model=model, params_dim=params_dim)

# Saving results in df_metric.
df_metric.loc['m4_randomforest'] = ['RandomForestClassifier', score[0], score[1], score[2], score[3], score[4]]
df_metric

,algorithm,accuracy,recall,precision,f2score,roc_auc
m1_baseline,NaN,0.346131,0.780185,0.221972,0.519100,NaN
m2_logregression,LogisticRegression,0.749750,0.615751,0.453154,0.573856,0.749058
m3_decisiontree,DecisionTree,0.771969,0.572225,0.492438,0.552066,0.757373
m4_randomforest,RandomForestClassifier,0.764132,0.617635,0.477426,0.582166,0.779022


## Results

In [17]:
# Best results.
df_metric.sort_values('roc_auc', ascending=False).round(3)

,algorithm,accuracy,recall,precision,f2score,roc_auc
m4_randomforest,RandomForestClassifier,0.764,0.618,0.477,0.582,0.779
m3_decisiontree,DecisionTree,0.772,0.572,0.492,0.552,0.757
m2_logregression,LogisticRegression,0.750,0.616,0.453,0.574,0.749
m1_baseline,probability_in_sample,0.346,0.780,0.222,0.519,NaN


----

## Appendix

### Boruta for feature selection

In [1]:
from boruta import BorutaPy
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

In [57]:
# Let's create a dummy dataframe.
df = pd.DataFrame({
    'feat_a_relevant': np.random.normal(2, 5, 100),
    'feat_b_relevant': np.random.normal(-3, 1, 100),
    'feat_c_irrelevant': np.random.normal(0, 5, 100)})

df['y'] = 2 * df['feat_a_relevant'] -  df['feat_b_relevant'] / 2 + np.random.normal(0, 2, 100)

print('''Features A and B are influentional to y, while C are not. It is expected that Boruto confirms this fact.''')

display(df.head())


Features A and B are influentional to y, while C are not. It is expected that Boruto confirm this fact.


,feat_a_relevant,feat_b_relevant,feat_c_irrelevant,y
0,0.147896,-2.678392,4.964442,2.832847
1,5.715697,-4.026181,5.624898,13.156988
2,-2.108079,-2.143536,-10.197006,-3.649758
3,8.859634,-4.175130,-3.449414,17.967550
4,1.394410,-1.418882,-3.751191,6.417268


In [58]:
df.corr()

,feat_a_relevant,feat_b_relevant,feat_c_irrelevant,y
feat_a_relevant,1.000000,-0.094358,-0.011781,0.988952
feat_b_relevant,-0.094358,1.000000,-0.068457,-0.118947
feat_c_irrelevant,-0.011781,-0.068457,1.000000,-0.019522
y,0.988952,-0.118947,-0.019522,1.000000


In [77]:
boruta = BorutaPy(
    estimator = RandomForestRegressor(max_depth = 5, random_state = 52, n_estimators=200),
    n_estimators = 'auto',
    max_iter = 100
    )

boruta.fit(np.array(df[['feat_a_relevant', 'feat_b_relevant', 'feat_c_irrelevant']]), np.array(df['y']))

BorutaPy(estimator=RandomForestRegressor(max_depth=5, n_estimators=40,
                                         random_state=RandomState(MT19937) at 0x1F220B62240),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x1F220B62240)

In [100]:
# Relevant features.
boruta.support_, boruta.ranking_

(array([ True, False, False]), array([1, 3, 2]))

In [94]:
# Undecided features.
boruta.support_weak_

array([False, False, False])

In [99]:
# Maybe rescaling the dataset helps?
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(df)
df_scaled = scaler.transform(df)

df_scaled[:,:3]
boruta.fit(df_scaled[:,:3], df_scaled[:,3])

BorutaPy(estimator=RandomForestRegressor(max_depth=5, n_estimators=40,
                                         random_state=RandomState(MT19937) at 0x1F220B62240),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x1F220B62240)

In [95]:
# Relevant features.
boruta.support_, boruta.ranking_

(array([ True, False, False]), array([1, 3, 2]))

In [98]:
print('Rescaling did not worked.')

Rescaling did not worked.


In [91]:
# Comparing Boruta with Recursive Feature Elimination;
from sklearn.feature_selection import RFE
rfe = RFE(RandomForestRegressor(max_depth = 5, random_state = 52, n_estimators=200), n_features_to_select=2, step=1)
rfe.fit(np.array(df[['feat_a_relevant', 'feat_b_relevant', 'feat_c_irrelevant']]), np.array(df['y']))

RFE(estimator=RandomForestRegressor(max_depth=5, n_estimators=200,
                                    random_state=52),
    n_features_to_select=2)

In [97]:
rfe.support_, rfe.ranking_

(array([ True, False,  True]), array([1, 2, 1]))

In [103]:
print('RFE gave the same result as Boruta. At least with Boruta setting a number of relevant features was not required.')

RFE gave the same result as Boruta. At least with Boruta setting a number of relevant features was not required.
